# Market impact modeling: Almgren-Chriss framework implementation

---

**Authors**: Grégoire Marguier & Pierre Robin-Schnepf  
**ENSAE Paris** - Python for Data Science (2025-2026)

---

## Table of contents

1. [Introduction & Problem Statement](#1-introduction)
2. [Data Collection & Exploration](#2-data)
3. [Model Parameter Calibration](#3-calibration)
4. [The Almgren-Chriss Model](#4-model)
5. [Results & Visualizations](#5-results)
6. [Conclusion](#6-conclusion)

---

<a id="1-introduction"></a>
## 1. Introduction & problem statement

### 1.1 Context

When an institutional investor wants to execute a large order in the financial market, they face a fundamental dilemma: **executing quickly leads to significant market impact** (the price moves unfavorably), while **executing slowly exposes them to volatility risk** (the market may move against them during execution).

This problem is at the heart of **market microstructure** and has major practical implications for fund managers, algorithmic traders, and market makers.

### 1.2 Research question

> **How can we model and optimize transaction costs related to market impact when executing large orders?**

More specifically:
- How can we quantify the temporary and permanent impact of an order on price?
- What is the optimal execution trajectory to minimize total costs?
- Which parameters influence this optimal strategy?

### 1.3 The Almgren-Chriss model (2001)

We implement the **Almgren and Chriss (2001)** model, a seminal reference in quantitative finance. This model provides an analytical solution to the optimal execution problem by minimizing:

$$\min_{v_t} \quad \mathbb{E}[\text{Cost}] + \lambda \cdot \text{Var}[\text{Cost}]$$

where $\lambda$ represents the investor's risk aversion.

### 1.4 Main contribution: empirical calibration

Our main contribution lies in the **empirical calibration of model parameters** using real data:
- Historical market data (stocks and cryptocurrencies)
- Real-time order book snapshots
- Impact coefficient estimation via non-linear regression

---

## Setup & imports

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# S3 access (for SSPCloud)
import s3fs

# Graphics configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Custom colors
COLORS = {
    'optimal': '#2E86AB',
    'twap': '#E94F37',
    'fees': '#F39C12',
    'temp': '#3498DB',
    'perm': '#E74C3C',
    'risk': '#9B59B6'
}

print("Configuration loaded successfully")

<a id="2-data"></a>
## 2. Data collection & exploration

### 2.1 Data sources

Our analysis relies on **multiple complementary data sources**:

| Source | Asset Type | Frequency | Period | Method |
|--------|------------|-----------|--------|--------|
| Databento/S3 | Stocks (AAPL, MSFT, GOOG) | 1 min OHLCV | Jan-Jun 2025 | S3 Transfer (from Databento) |
| Binance API | Crypto (BTC, ETH, SOL) | 1 min OHLCV | November 2025 | REST API |
| Binance API | Order Books | Real-time | On demand | API `/api/v3/depth` |

In [ ]:
# Load market parameters from S3 (with local fallback)
import s3fs

BUCKET = 'gmarguier'
S3_PREFIX = 'market-impact-data/processed'

def load_from_s3_or_local(s3_path, local_path):
    """Try to load from S3 first, then fall back to local file"""
    try:
        s3 = s3fs.S3FileSystem(
            client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
        )
        with s3.open(s3_path, 'rb') as f:
            df = pd.read_parquet(f)
        print(f"✓ Loaded from S3: {s3_path}")
        return df
    except Exception as e:
        print(f"⚠ S3 not available ({e}), trying local file...")
        try:
            df = pd.read_parquet(local_path)
            print(f"✓ Loaded from local: {local_path}")
            return df
        except FileNotFoundError:
            print(f"✗ Local file not found: {local_path}")
            return None

# Load market parameters
df_params = load_from_s3_or_local(
    s3_path=f'{BUCKET}/{S3_PREFIX}/market_parameters.parquet',
    local_path='../data/processed/market_parameters.parquet'
)

if df_params is not None:
    print("\n" + "=" * 70)
    print("ESTIMATED MARKET PARAMETERS")
    print("=" * 70)

    # Formatted display
    display_cols = ['symbol', 'asset_type', 'vol_annual', 'volume_per_day', 'S0_recent', 'spread_bps_mean']
    df_display = df_params[display_cols].copy()
    df_display.columns = ['Symbol', 'Type', 'Volatility (ann.)', 'Volume/day', 'Price', 'Spread (bps)']
    df_display['Volatility (ann.)'] = df_display['Volatility (ann.)'].apply(lambda x: f"{x*100:.1f}%")
    df_display['Volume/day'] = df_display['Volume/day'].apply(lambda x: f"{x:,.0f}")
    df_display['Price'] = df_display['Price'].apply(lambda x: f"${x:,.2f}")
    df_display['Spread (bps)'] = df_display['Spread (bps)'].apply(lambda x: f"{x:.2f}")

    print(df_display.to_string(index=False))
else:
    print("\n⚠ No market parameters available. Please run 01_data_collection.ipynb first.")

### 2.2 Market data exploration

In [ ]:
# Load crypto data from S3 (with local fallback)
def load_parquet_from_s3_or_local(s3_path, local_path):
    """Try to load parquet from S3 first, then fall back to local file"""
    try:
        s3 = s3fs.S3FileSystem(
            client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
        )
        with s3.open(s3_path, 'rb') as f:
            df = pd.read_parquet(f)
        print(f"✓ Loaded from S3: {s3_path.split('/')[-1]}")
        return df
    except Exception:
        try:
            df = pd.read_parquet(local_path)
            print(f"✓ Loaded from local: {local_path.split('/')[-1]}")
            return df
        except FileNotFoundError:
            return None

df_btc = load_parquet_from_s3_or_local(
    s3_path=f'{BUCKET}/{S3_PREFIX}/crypto/BTCUSDT_1m.parquet',
    local_path='../data/processed/crypto/BTCUSDT_1m.parquet'
)

if df_btc is not None:
    print(f"\nBTCUSDT data loaded:")
    print(f"   Period: {df_btc['timestamp'].min()} -> {df_btc['timestamp'].max()}")
    print(f"   Observations: {len(df_btc):,}")
    print(f"   Average price: ${df_btc['close'].mean():,.2f}")
else:
    print("⚠ BTCUSDT data not available")

In [ ]:
# Price evolution visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# BTC Price
ax = axes[0, 0]
ax.plot(df_btc['timestamp'], df_btc['close'], linewidth=0.8, color=COLORS['optimal'])
ax.set_title('BTC/USDT Price Evolution (November 2025)', fontweight='bold')
ax.set_ylabel('Price ($)')
ax.grid(True, alpha=0.3)

# Returns distribution
ax = axes[0, 1]
returns = df_btc['close'].pct_change().dropna()
ax.hist(returns * 100, bins=100, edgecolor='white', alpha=0.7, color=COLORS['optimal'])
ax.axvline(0, color='red', linestyle='--', linewidth=1.5)
ax.set_title('Returns Distribution (1 min)', fontweight='bold')
ax.set_xlabel('Return (%)')
ax.set_ylabel('Frequency')
ax.set_xlim(-2, 2)

# Average intraday volume
ax = axes[1, 0]
df_btc['hour'] = pd.to_datetime(df_btc['timestamp']).dt.hour
hourly_volume = df_btc.groupby('hour')['volume'].mean()
ax.bar(hourly_volume.index, hourly_volume.values, color=COLORS['temp'], edgecolor='white')
ax.set_title('Average Hourly Volume Profile (BTC)', fontweight='bold')
ax.set_xlabel('Hour (UTC)')
ax.set_ylabel('Average Volume (BTC)')

# Volatility comparison by asset
ax = axes[1, 1]
volatilities = df_params.set_index('symbol')['vol_annual'] * 100
colors = [COLORS['optimal'] if t == 'crypto' else COLORS['twap'] 
          for t in df_params.set_index('symbol')['asset_type']]
bars = ax.bar(volatilities.index, volatilities.values, color=colors, edgecolor='white')
ax.set_title('Annualized Volatility by Asset', fontweight='bold')
ax.set_ylabel('Volatility (%)')
ax.axhline(50, color='gray', linestyle='--', alpha=0.5, label='50%')

plt.tight_layout()
plt.savefig('../results/figures/01_data_exploration.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n Figure saved: results/figures/01_data_exploration.png")

### 2.3 Order book analysis

The order book allows us to understand the **available liquidity** at different price levels and estimate **slippage** when executing orders of varying sizes.

In [ ]:
# Load calibrated parameters from S3 or local (with default values as fallback)
def load_json_from_s3_or_local(s3_path, local_path):
    """Try to load JSON from S3 first, then fall back to local file"""
    try:
        s3 = s3fs.S3FileSystem(
            client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
        )
        with s3.open(s3_path, 'r') as f:
            data = json.load(f)
        print(f"✓ Calibrated parameters loaded from S3")
        return data
    except Exception:
        try:
            with open(local_path, 'r') as f:
                data = json.load(f)
            print(f"✓ Calibrated parameters loaded from local file")
            return data
        except FileNotFoundError:
            return None

calibrated = load_json_from_s3_or_local(
    s3_path=f'{BUCKET}/{S3_PREFIX.replace("/processed", "")}/results/calibrated_parameters.json',
    local_path='../results/calibrated_parameters.json'
)

if calibrated is not None:
    print(f"   Symbol: {calibrated['symbol']}")
    print(f"   Snapshots used: {calibrated['num_snapshots']}")
    calibrated_available = True
else:
    # Default values based on typical Binance calibration
    print("⚠ Calibrated parameters not available, using default values")
    calibrated = {
        'symbol': 'BTCUSDT',
        'daily_volume': 26000,
        'num_snapshots': 360,
        'quadratic_model': {
            'parameters': {'psi': 0.001, 'eta': 0.1, 'k': 1e-6},
            'goodness_of_fit': {'r_squared': 0.95}
        },
        'powerlaw_model': {
            'parameters': {'psi': 0.001, 'eta': 0.01, 'k': 1e-6},
            'goodness_of_fit': {'r_squared': 0.89}
        }
    }
    calibrated_available = True
    print(f"   Using default parameters for {calibrated['symbol']}")

<a id="3-calibration"></a>
## 3. Model parameter calibration

### 3.1 Execution cost structure

The Almgren-Chriss model decomposes costs into several components:

$$\text{Total Cost} = \underbrace{\psi \cdot |q_0|}_{\text{fixed fees}} + \underbrace{\int_0^T \eta \cdot \frac{v_t^2}{V} \, dt}_{\text{temporary impact}} + \underbrace{\frac{1}{2} \gamma \cdot q_0^2}_{\text{permanent impact}}$$

where:
- $q_0$: quantity to execute
- $v_t = \dot{q}_t$: execution speed
- $V$: average daily volume
- $\gamma$: permanent impact coefficient (in $/unit), with $k = \gamma / S_0$ its dimensionless version

### 3.2 Calibration method

| Parameter | Estimable? | Method |
|-----------|------------|--------|
| **ψ** (fixed fees) | YES | Binance fees = 10 bps (known) |
| **η** (temp. impact) | YES | Regression on slippage data |
| **φ** (exponent) | NO | Fixed to 0.5 (literature) or 1.0 (AC2001) |
| **γ or k** (perm. impact) | PARTIAL | AC2001 rule: 10% of volume → 1 spread |

In [ ]:
if calibrated_available:
    print("=" * 70)
    print("CALIBRATED PARAMETERS")
    print("=" * 70)
    
    # Quadratic model (original AC2001)
    quad = calibrated['quadratic_model']['parameters']
    quad_fit = calibrated['quadratic_model']['goodness_of_fit']
    
    # Power-law model
    power = calibrated['powerlaw_model']['parameters']
    power_fit = calibrated['powerlaw_model']['goodness_of_fit']
    
    print("\n┌─────────────────────────────────────────────────────────────────┐")
    print("│                    QUADRATIC MODEL (φ = 1)                      │")
    print("├─────────────────────────────────────────────────────────────────┤")
    print(f"│  ψ (fixed fees)       = {quad['psi']:.6f}  ({quad['psi']*10000:.0f} bps)              │")
    print(f"│  η (temp. impact)     = {quad['eta']:.4f}                                │")
    print(f"│  k (perm. impact)     = {quad['k']:.2e}                          │")
    print(f"│  R² = {quad_fit['r_squared']:.4f}                                           │")
    print("└─────────────────────────────────────────────────────────────────┘")
    
    print("\n┌─────────────────────────────────────────────────────────────────┐")
    print("│                    POWER-LAW MODEL (φ = 0.5)                    │")
    print("├─────────────────────────────────────────────────────────────────┤")
    print(f"│  ψ (fixed fees)       = {power['psi']:.6f}  ({power['psi']*10000:.0f} bps)              │")
    print(f"│  η (temp. impact)     = {power['eta']:.6f}                              │")
    print(f"│  k (perm. impact)     = {power['k']:.2e}                          │")
    print(f"│  R² = {power_fit['r_squared']:.4f}                                           │")
    print("└─────────────────────────────────────────────────────────────────┘")
    
    print("\n Key insight:")
    print(f"   On Binance, transaction fees ({quad['psi']*10000:.0f} bps) >> spread (~0 bps)")
    print("   -> Fixed cost ψ is dominated by fees, not spread.")

In [ ]:
# Calibration visualization
if calibrated_available:
    # Recreate slippage data for visualization
    rho_values = np.array([0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]) / calibrated['daily_volume']
    rho_smooth = np.linspace(rho_values.min(), rho_values.max(), 100)
    
    # Observed slippage (approximation based on parameters)
    slippage_obs_quad = quad['eta'] * rho_values
    slippage_obs_power = power['eta'] * np.power(rho_values, 0.5)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Plot 1: Model comparison
    ax = axes[0]
    ax.plot(rho_smooth * 100, power['eta'] * np.power(rho_smooth, 0.5) * 10000, 
            '-', linewidth=2.5, color=COLORS['optimal'], label=f'Power-law (φ=0.5)')
    ax.plot(rho_smooth * 100, quad['eta'] * rho_smooth * 10000, 
            '--', linewidth=2.5, color=COLORS['twap'], label=f'Quadratic (φ=1)')
    ax.set_xlabel('Participation rate ρ (%)')
    ax.set_ylabel('Slippage (bps)')
    ax.set_title('Cost Model Comparison', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Plot 2: Cost decomposition
    ax = axes[1]
    rho_demo = np.linspace(0.001, 0.05, 100)  # 0.1% to 5%
    
    cost_fees = np.ones_like(rho_demo) * quad['psi'] * 10000
    cost_temp = quad['eta'] * rho_demo * 10000
    cost_perm = quad['k'] * rho_demo * 10000 * calibrated['daily_volume']
    
    ax.fill_between(rho_demo * 100, 0, cost_fees, 
                    alpha=0.7, label=f'Fees ψ = {quad["psi"]*10000:.0f} bps', color=COLORS['fees'])
    ax.fill_between(rho_demo * 100, cost_fees, cost_fees + cost_temp, 
                    alpha=0.7, label='Temporary impact', color=COLORS['temp'])
    ax.fill_between(rho_demo * 100, cost_fees + cost_temp, cost_fees + cost_temp + cost_perm, 
                    alpha=0.7, label='Permanent impact', color=COLORS['perm'])
    ax.set_xlabel('Participation rate ρ (%)')
    ax.set_ylabel('Cost (bps)')
    ax.set_title('Cost Decomposition (Quadratic Model)', fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('../results/figures/02_calibration.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n Figure saved: results/figures/02_calibration.png")

<a id="4-model"></a>
## 4. The Almgren-Chriss model

### 4.1 Mathematical formulation

**Price dynamics** (with permanent impact):
$$dS_t = \sigma \, dW_t + b \cdot v_t \, dt$$

where $v_t = \dot{q}_t$ is the execution speed and $b$ is the permanent impact coefficient.

**Optimization objective**:
$$\min_{v_t} \quad \mathbb{E}\left[ X_T \right] + \frac{\lambda}{2} \operatorname{Var}\left[ X_T \right]$$

### 4.2 Analytical solution

The optimal trajectory admits a **closed-form solution**:

$$\boxed{q^*(t) = q_0 \frac{\sinh \left( \kappa (T - t) \right)}{\sinh (\kappa T)}}$$

with the **urgency parameter**:

$$\boxed{\kappa = \sqrt{\frac{\lambda \sigma^2 V}{2 \eta}}}$$

**Interpretation of κT**:
- $\kappa T < 1$: Slow execution (close to TWAP)
- $\kappa T \approx 1-3$: Intermediate regime (significant optimization)
- $\kappa T > 5$: Fast execution (market impact dominates)

In [ ]:
class AlmgrenChrissQuadratic:
    """
    Almgren-Chriss (2001) model with quadratic costs.
    Closed-form analytical solution.
    """
    
    def __init__(self, lambda_risk, sigma, k, eta, V, psi=0.0, S0=1.0):
        self.lambda_risk = lambda_risk
        self.sigma = sigma
        self.k = k
        self.eta = eta
        self.V = V
        self.psi = psi
        self.S0 = S0
    
    def compute_kappa(self):
        """Urgency parameter κ = sqrt(λσ²V / 2η)"""
        return np.sqrt(self.lambda_risk * self.sigma**2 * self.V / (2 * self.eta))
    
    def optimal_trajectory(self, q0, T, N=390):
        """Optimal trajectory q*(t) and speed v*(t)"""
        kappa = self.compute_kappa()
        t = np.linspace(0, T, N+1)
        
        if kappa * T > 100:
            q_star = q0 * np.exp(-kappa * t)
            v_star = -q0 * kappa * np.exp(-kappa * t)
        elif kappa * T < 0.01:
            q_star = q0 * (1 - t / T)
            v_star = -q0 / T * np.ones_like(t)
        else:
            q_star = q0 * np.sinh(kappa * (T - t)) / np.sinh(kappa * T)
            v_star = q0 * (-kappa) * np.cosh(kappa * (T - t)) / np.sinh(kappa * T)
        
        return t, q_star, v_star
    
    def compute_costs(self, q0, T, N, strategy='optimal'):
        """Compute execution costs"""
        if strategy == 'optimal':
            t, q, v = self.optimal_trajectory(q0, T, N)
        else:  # TWAP
            t = np.linspace(0, T, N+1)
            q = q0 * (1 - t / T)
            v = -(q0 / T) * np.ones_like(t)
        
        dt = T / N
        
        # 1. Transaction fees
        transaction_fees = self.psi * abs(q0)
        
        # 2. Temporary impact
        execution_cost = np.sum(self.eta * v[:-1]**2 / self.V * dt)
        
        # 3. Permanent impact
        permanent_impact = self.k * q0**2 / 2
        
        # 4. Timing risk
        timing_risk = (self.lambda_risk / 2) * self.sigma**2 * np.sum(q[:-1]**2 * dt)
        
        return {
            'transaction_fees': transaction_fees,
            'execution_cost': execution_cost,
            'permanent_impact': permanent_impact,
            'timing_risk': timing_risk,
            'total_cost': transaction_fees + execution_cost + permanent_impact + timing_risk
        }

print("AlmgrenChrissQuadratic class defined")

<a id="5-results"></a>
## 5. Results & visualizations

### 5.1 Application to crypto market (BTCUSDT)

In [ ]:
# Market parameters for BTCUSDT
params_btc = df_params[df_params['symbol'] == 'BTCUSDT'].iloc[0]

sigma_btc = params_btc['vol_annual']
V_btc = params_btc['volume_per_day']
S0_btc = params_btc['S0_recent']

# Calibrated parameters
if calibrated_available:
    psi_btc = calibrated['quadratic_model']['parameters']['psi']
    eta_btc = calibrated['quadratic_model']['parameters']['eta']
    k_btc = calibrated['quadratic_model']['parameters']['k']
else:
    psi_btc = 0.001
    eta_btc = 0.1
    k_btc = 1e-6

# Execution scenario
participation = 0.05  # 5% of daily volume
q0_btc = participation * V_btc
T_btc = 1.0  # 1 day
N_btc = 1440  # 24h in minutes
lambda_risk = 1e-4  # High risk aversion

print("=" * 70)
print("EXECUTION SCENARIO - BTCUSDT")
print("=" * 70)
print(f"\n Market parameters:")
print(f"   σ (volatility)      = {sigma_btc*100:.1f}% (annualized)")
print(f"   V (volume/day)      = {V_btc:,.0f} BTC")
print(f"   S₀ (price)          = ${S0_btc:,.2f}")
print(f"\n Model parameters:")
print(f"   ψ (fees)            = {psi_btc*10000:.0f} bps")
print(f"   η (temp. impact)    = {eta_btc:.4f}")
print(f"   k (perm. impact)    = {k_btc:.2e}")
print(f"   λ (risk aversion)   = {lambda_risk:.0e}")
print(f"\n Order to execute:")
print(f"   Participation       = {participation*100:.0f}% of daily volume")
print(f"   Quantity            = {q0_btc:,.0f} BTC")
print(f"   Notional            = ${q0_btc * S0_btc:,.0f}")
print(f"   Horizon             = {T_btc} day ({N_btc} minutes)")

In [ ]:
# Create model and compute trajectories
model_btc = AlmgrenChrissQuadratic(
    lambda_risk=lambda_risk,
    sigma=sigma_btc,
    k=k_btc,
    eta=eta_btc,
    V=V_btc,
    psi=psi_btc,
    S0=S0_btc
)

# Trajectories
t_opt, q_opt, v_opt = model_btc.optimal_trajectory(q0_btc, T_btc, N_btc)
t_twap = np.linspace(0, T_btc, N_btc+1)
q_twap = q0_btc * (1 - t_twap / T_btc)
v_twap = -(q0_btc / T_btc) * np.ones_like(t_twap)

# Costs
costs_opt = model_btc.compute_costs(q0_btc, T_btc, N_btc, 'optimal')
costs_twap = model_btc.compute_costs(q0_btc, T_btc, N_btc, 'twap')

kappa = model_btc.compute_kappa()
print(f"\n Urgency parameter: κ = {kappa:.4f}, κT = {kappa * T_btc:.3f}")

In [ ]:
# Main visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Liquidation trajectory
ax = axes[0, 0]
ax.plot(t_opt * 24, q_opt, linewidth=2.5, color=COLORS['optimal'], label='Optimal')
ax.plot(t_twap * 24, q_twap, '--', linewidth=2, color=COLORS['twap'], label='TWAP')
ax.fill_between(t_opt * 24, q_opt, alpha=0.2, color=COLORS['optimal'])
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Position (BTC)')
ax.set_title('Liquidation Trajectory', fontweight='bold', fontsize=13)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)

# 2. Execution speed
ax = axes[0, 1]
ax.plot(t_opt * 24, -v_opt, linewidth=2.5, color=COLORS['optimal'], label='Optimal')
ax.axhline(-v_twap[0], color=COLORS['twap'], linestyle='--', linewidth=2, label='TWAP')
ax.set_xlabel('Time (hours)')
ax.set_ylabel('Execution Speed (BTC/day)')
ax.set_title('Execution Speed', fontweight='bold', fontsize=13)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)

# 3. Cost decomposition (bars)
ax = axes[1, 0]
notional = q0_btc * S0_btc
components = ['Fees', 'Temp. Impact', 'Perm. Impact', 'Timing Risk']
costs_opt_bps = [
    costs_opt['transaction_fees'] / q0_btc * 10000,
    costs_opt['execution_cost'] / q0_btc * 10000,
    costs_opt['permanent_impact'] / q0_btc * 10000,
    costs_opt['timing_risk'] / q0_btc * 10000
]
costs_twap_bps = [
    costs_twap['transaction_fees'] / q0_btc * 10000,
    costs_twap['execution_cost'] / q0_btc * 10000,
    costs_twap['permanent_impact'] / q0_btc * 10000,
    costs_twap['timing_risk'] / q0_btc * 10000
]

x = np.arange(len(components))
width = 0.35
bars1 = ax.bar(x - width/2, costs_opt_bps, width, label='Optimal', color=COLORS['optimal'])
bars2 = ax.bar(x + width/2, costs_twap_bps, width, label='TWAP', color=COLORS['twap'])
ax.set_ylabel('Cost (bps)')
ax.set_title('Cost Decomposition', fontweight='bold', fontsize=13)
ax.set_xticks(x)
ax.set_xticklabels(components)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# 4. Total comparison
ax = axes[1, 1]
total_opt = costs_opt['total_cost'] / q0_btc * 10000
total_twap = costs_twap['total_cost'] / q0_btc * 10000
gain = total_twap - total_opt
gain_pct = gain / total_twap * 100

bars = ax.bar(['Optimal', 'TWAP'], [total_opt, total_twap], 
              color=[COLORS['optimal'], COLORS['twap']], edgecolor='white', linewidth=2)

# Annotations
for bar, val in zip(bars, [total_opt, total_twap]):
    ax.annotate(f'{val:.1f} bps', xy=(bar.get_x() + bar.get_width()/2, val),
                xytext=(0, 5), textcoords='offset points', ha='center', fontsize=12, fontweight='bold')

ax.annotate(f'Savings: {gain:.1f} bps\n({gain_pct:.1f}%)', 
            xy=(0.5, (total_opt + total_twap)/2), fontsize=14, fontweight='bold',
            ha='center', color='green',
            bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

ax.set_ylabel('Total Cost (bps)')
ax.set_title('Optimal Strategy vs TWAP Comparison', fontweight='bold', fontsize=13)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../results/figures/03_results_btc.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n Figure saved: results/figures/03_results_btc.png")

### 5.2 Sensitivity analysis

In [ ]:
# Sensitivity analysis across different scenarios
scenarios = [
    {'name': 'Low urgency (T=5d, low λ)', 'T': 5.0, 'lambda_risk': 1e-6},
    {'name': 'Medium urgency (T=1d, med λ)', 'T': 1.0, 'lambda_risk': 1e-5},
    {'name': 'High urgency (T=1d, high λ)', 'T': 1.0, 'lambda_risk': 1e-4},
    {'name': 'Very urgent (T=2h, high λ)', 'T': 2/24, 'lambda_risk': 1e-4},
]

results = []
for scenario in scenarios:
    T = scenario['T']
    lam = scenario['lambda_risk']
    N = max(int(T * 1440), 10)
    
    model = AlmgrenChrissQuadratic(lam, sigma_btc, k_btc, eta_btc, V_btc, psi=psi_btc)
    
    costs_opt = model.compute_costs(q0_btc, T, N, 'optimal')
    costs_twap = model.compute_costs(q0_btc, T, N, 'twap')
    
    opt_bps = costs_opt['total_cost'] / q0_btc * 10000
    twap_bps = costs_twap['total_cost'] / q0_btc * 10000
    kappa = model.compute_kappa()
    
    results.append({
        'Scenario': scenario['name'],
        'T (hours)': T * 24,
        'κT': kappa * T,
        'Optimal (bps)': opt_bps,
        'TWAP (bps)': twap_bps,
        'Savings (bps)': twap_bps - opt_bps,
        'Savings (%)': (twap_bps - opt_bps) / twap_bps * 100 if twap_bps > 0 else 0
    })

df_results = pd.DataFrame(results)

print("=" * 90)
print("SENSITIVITY ANALYSIS")
print("=" * 90)
print(f"\n{df_results.to_string(index=False)}")

print("\n Interpretation:")
print("   • κT < 1: Slow execution recommended (close to TWAP)")
print("   • κT ~ 1-3: Intermediate regime (significant optimization)")
print("   • κT > 5: Fast execution (market impact dominates)")

In [ ]:
# Sensitivity analysis visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Costs by scenario
ax = axes[0]
x = np.arange(len(df_results))
width = 0.35
ax.bar(x - width/2, df_results['Optimal (bps)'], width, label='Optimal', color=COLORS['optimal'])
ax.bar(x + width/2, df_results['TWAP (bps)'], width, label='TWAP', color=COLORS['twap'])
ax.set_ylabel('Cost (bps)')
ax.set_title('Costs by Scenario', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(['Low', 'Medium', 'High', 'Very High'], rotation=0)
ax.set_xlabel('Urgency Level')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Optimization gain vs κT
ax = axes[1]
ax.scatter(df_results['κT'], df_results['Savings (%)'], s=150, c=COLORS['optimal'], edgecolors='white', linewidth=2, zorder=5)
ax.plot(df_results['κT'], df_results['Savings (%)'], '--', alpha=0.5, color=COLORS['optimal'])

# Annotations
for i, row in df_results.iterrows():
    ax.annotate(f"{row['Savings (%)']:.1f}%", 
                xy=(row['κT'], row['Savings (%)']),
                xytext=(5, 5), textcoords='offset points', fontsize=10)

ax.axvline(1, color='gray', linestyle=':', alpha=0.5, label='κT = 1')
ax.axvline(3, color='gray', linestyle=':', alpha=0.5, label='κT = 3')
ax.set_xlabel('Urgency Parameter κT')
ax.set_ylabel('Optimization Savings (%)')
ax.set_title('Impact of Urgency Parameter', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/04_sensitivity.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n Figure saved: results/figures/04_sensitivity.png")

### 5.3 Stocks vs crypto comparison

In [ ]:
# Try to load stock data from S3 or local
def check_stocks_available():
    """Check if stock data is available from S3 or locally"""
    try:
        s3 = s3fs.S3FileSystem(
            client_kwargs={'endpoint_url': 'https://minio.lab.sspcloud.fr'}
        )
        s3_path = f'{BUCKET}/{S3_PREFIX}/stocks/all_stocks_1m.parquet'
        if s3.exists(s3_path):
            return 's3', s3_path
    except:
        pass
    
    local_path = '../data/processed/stocks/all_stocks_1m.parquet'
    if os.path.exists(local_path):
        return 'local', local_path
    
    return None, None

stocks_source, stocks_path = check_stocks_available()
stocks_available = stocks_source is not None

if stocks_available:
    # AAPL parameters
    params_aapl = df_params[df_params['symbol'] == 'AAPL'].iloc[0]
    sigma_aapl = params_aapl['vol_annual']
    V_aapl = params_aapl['volume_per_day']
    S0_aapl = params_aapl['S0_recent']
    
    # Literature parameters for stocks
    eta_aapl = 0.10
    k_aapl = 5e-8
    psi_aapl = 0.0001  # 1 bps (spread)
    
    # Comparable scenario
    participation_aapl = 0.05
    q0_aapl = participation_aapl * V_aapl
    T_aapl = 1.0
    N_aapl = 390  # 6.5h
    
    model_aapl = AlmgrenChrissQuadratic(1e-5, sigma_aapl, k_aapl, eta_aapl, V_aapl, psi=psi_aapl)
    
    costs_opt_aapl = model_aapl.compute_costs(q0_aapl, T_aapl, N_aapl, 'optimal')
    costs_twap_aapl = model_aapl.compute_costs(q0_aapl, T_aapl, N_aapl, 'twap')
    
    opt_bps_aapl = costs_opt_aapl['total_cost'] / q0_aapl * 10000
    twap_bps_aapl = costs_twap_aapl['total_cost'] / q0_aapl * 10000
    
    print("=" * 70)
    print("STOCKS vs CRYPTO COMPARISON")
    print("=" * 70)
    print(f"(Stock data loaded from {stocks_source})")
    
    comparison_data = {
        'Metric': ['Volatility (ann.)', 'Volume/day ($)', 'Notional ($)', 
                   'Optimal Cost (bps)', 'TWAP Cost (bps)', 'Optimization Savings (%)'],
        'AAPL (Stock)': [
            f"{sigma_aapl*100:.1f}%",
            f"${V_aapl * S0_aapl:,.0f}",
            f"${q0_aapl * S0_aapl:,.0f}",
            f"{opt_bps_aapl:.1f}",
            f"{twap_bps_aapl:.1f}",
            f"{(twap_bps_aapl - opt_bps_aapl)/twap_bps_aapl*100:.1f}%"
        ],
        'BTCUSDT (Crypto)': [
            f"{sigma_btc*100:.1f}%",
            f"${V_btc * S0_btc:,.0f}",
            f"${q0_btc * S0_btc:,.0f}",
            f"{total_opt:.1f}",
            f"{total_twap:.1f}",
            f"{gain_pct:.1f}%"
        ]
    }
    
    df_comparison = pd.DataFrame(comparison_data)
    print(f"\n{df_comparison.to_string(index=False)}")
else:
    print("Stock data not available (run from SSPCloud or run 01_data_collection.ipynb first)")
    print("   Comparison limited to cryptocurrencies.")

<a id="6-conclusion"></a>
## 6. Conclusion

### 6.1 Key results

In [ ]:
print("=" * 70)
print("RESULTS SUMMARY")
print("=" * 70)

print("""
 DATA COLLECTION
---------------------------------------------------------------------
• 6 assets analyzed: 3 stocks (AAPL, MSFT, GOOG) + 3 cryptos (BTC, ETH, SOL)
• Minute-level data over 1-6 months
• 360+ order book snapshots collected
• Multiple sources: Databento (S3), Binance API

 CALIBRATION
---------------------------------------------------------------------
• Quadratic model (AC2001): R² = 0.995
• Power-law model (φ=0.5): R² = 0.893
• On Binance: fees (10 bps) >> spread (~0 bps)

 MODELING
---------------------------------------------------------------------
• Full implementation of the Almgren-Chriss model
• Analytical solution for quadratic costs
• TWAP vs optimal trajectory comparison

 KEY INSIGHTS
---------------------------------------------------------------------
• Optimization provides the most value in "medium urgency" regime (κT ~ 1-3)
• On liquid crypto, optimization savings are 10-15% vs TWAP
• Transaction fees dominate costs for small participation rates
""")

### 6.2 Limitations & future work

**Limitations of our approach:**
- Constant volatility model (fixed σ)
- Market volume assumed constant (fixed V)
- Permanent impact estimated via heuristic rule (not empirically calibrated)
- No differentiation between maker/taker fees

**Future improvements:**
- Integrate intraday volume profile (VWAP)
- Model stochastic volatility
- Calibrate permanent impact on meta-order data
- Compare with reinforcement learning strategies

### 6.3 References

1. **Almgren, R., & Chriss, N.** (2001). *Optimal execution of portfolio transactions*. Journal of Risk, 3, 5-40.

2. **Kyle, A. S.** (1985). *Continuous auctions and insider trading*. Econometrica, 53(6), 1315-1335.

3. **Gatheral, J.** (2010). *No-dynamic-arbitrage and market impact*. Quantitative Finance, 10(7), 749-759.

4. **Cartea, Á., Jaimungal, S., & Penalva, J.** (2015). *Algorithmic and High-Frequency Trading*. Cambridge University Press.

---

**End of Report**

*Notebook generated: December 2025*